In [1]:
import open3d as o3d
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import transforms3d
import copy

In [2]:
def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

In [29]:
# static_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/testing/testing_data/_corr_static/18.npy"
# dynamic_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/testing/testing_data/_corr_dynamic/18.npy"

# dynamic_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/d/emptycities_testd15.npy"
# static_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/rs/emptycities_testrs15.npy"

# dynamic_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/dynamic_out_npy/2.npy"
# static_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/static_out_npy/2.npy"

###################
dynamic_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/segmentation_preprocess/dslrseg_test/dslrseg_testd15.npy"
static_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/segmentation_preprocess/dslrseg_test/dslrseg_tests15.npy"

# dynamic_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/segmentation_preprocess_bu/dynamic_preprocessed/16.npy"
# static_npy = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/segmentation_preprocess_bu/static_preprocessed/16.npy"


print("starting")
static_arr = np.load(static_npy)
print("static done")
dynamic_arr = np.load(dynamic_npy)
print("dynamic done")

starting
static done
dynamic done


In [30]:
static_arr = from_polar_np(static_arr).transpose(0,2,3,1)
dynamic_arr = from_polar_np(dynamic_arr).transpose(0,2,3,1)

img_size = static_arr.shape
static_arr = np.concatenate([static_arr, np.zeros((img_size[0],img_size[1],img_size[2],1))], axis=-1)
img_size = dynamic_arr.shape
dynamic_arr = np.concatenate([dynamic_arr, np.zeros((img_size[0],img_size[1],img_size[2],1))], axis=-1)

In [31]:
static_arr.shape

(1274, 64, 1024, 4)

In [32]:
dynamic_arr.shape

(1274, 64, 1024, 4)

In [33]:
static_img = static_arr[-1]
# static_img = static_arr[2033]
# dynamic_img = dynamic_arr[2033]
# dynamic_img = dynamic_arr[50,5:45,::2]
# dynamic_img = dynamic_arr[50,:,::2]
dynamic_img = dynamic_arr[-1]



In [34]:
set((static_img == dynamic_img).reshape(1,-1)[0])

{False, True}

In [35]:
pd.DataFrame(np.abs((static_img - dynamic_img).reshape(1,-1)[0])).describe()

,0
count,262144.000000
mean,0.002191
std,0.014189
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,0.404000


In [36]:
static_pcd = o3d.geometry.PointCloud()
static_arr = static_img.reshape(-1,4)[:,:3]
static_arr = np.concatenate((static_arr[:,1:2], -static_arr[:,0:1], static_arr[:,2:3]),axis=-1)
static_pcd.points = o3d.utility.Vector3dVector(static_arr)

dynamic_pcd = o3d.geometry.PointCloud()
dynamic_arr = dynamic_img.reshape(-1,4)[:,:3]
dynamic_arr = np.concatenate((dynamic_arr[:,1:2], -dynamic_arr[:,0:1], dynamic_arr[:,2:3]),axis=-1)
dynamic_pcd.points = o3d.utility.Vector3dVector(dynamic_arr)

In [37]:
# Function to get transformation matrix for a given pose
def pose2matrix(translation_list, rotation_angle_list, zoom_list=[1,1,1]):
    # trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom_vec = np.array(zoom_list)
    # transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom_vec)
    transform_mat = transforms3d.affines.compose(translation_list, rot_mat, zoom_list)
    return transform_mat

def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([0, 0, 0])  # black source
    dst_pcd_tmp.paint_uniform_color([1, 0, 0])  # red target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

In [38]:
draw_registration_result(static_pcd, dynamic_pcd, 0, 0, 0)
# static is black
# dynamic is red

JVisualizer with 2 geometries